In [19]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("HUGGING_FACE_API_KEY")

In [20]:
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [21]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=api_key)


def generate_text(prompt, model="meta-llama/Llama-3.2-3B-Instruct", max_tokens=500):
	messages = [
		{
			"role": "user",
			"content": prompt
		}
	]

	stream = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            stream=True
	)

	response_text = ""
	for chunk in stream:
		response_text += chunk.choices[0].delta.content
	return response_text

In [23]:
countries = ["France", "Japan", "Australia"]
for country in countries:
    question = f"What is the capital of {country}?"
    response = generate_text(question)
    print(f"Q: {question}")
    print(f"A: {response}\n")

Q: What is the capital of France?
A: The capital of France is Paris.

Q: What is the capital of Japan?
A: The capital of Japan is Tokyo.

Q: What is the capital of Australia?
A: The capital of Australia is Canberra.



In [29]:
prompt = PromptTemplate(
    input_variables=["country"],
    template="What is the capital of {country}?"
)

def chain_invoke_api(prompt, input_var):
    formatted_prompt = prompt.format(**input_var)
    return generate_text(formatted_prompt)


countries = ["Sweden", "Mexico", "Vietnam"]
for country in countries:
    response = chain_invoke_api(prompt, {"country": country})
    print(f"Q: {prompt.format(country=country)}")
    print(f"A: {response}\n")

# Example 3: Sequential Chains using API
pt1 = PromptTemplate(
    input_variables=["topic"],
    template="Generate a random question about {topic}: Question: "
)
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}"
)

Q: What is the capital of Sweden?
A: The capital of Sweden is Stockholm.

Q: What is the capital of Mexico?
A: The capital of Mexico is Mexico City (also known as Ciudad de México). It is the largest city in Mexico and the country's political, economic, and cultural center.

Q: What is the capital of Vietnam?
A: The capital of Vietnam is Hanoi.



In [32]:
pdf = './pdf/what-is-generative-ai.pdf'
loaders = [PyPDFLoader(pdf)]

index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
).from_loaders(loaders)

C:\Users\Albert\AppData\Local\Temp\ipykernel_16868\3791850439.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),
c:\Users\Albert\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\indexes\vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [ ]:

chain = RetrievalQA.from_chain_type(
    llm=generate_text,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    input_key="question"
)

In [ ]:
question = "What is Machine Learning?"
response = chain.invoke({"question": question})
print(f"Q: {question}")
print(f"A: {response['result']}")